In [1]:
import pandas as pd
import numpy as np
import keras
from keras.layers import Dense, GlobalAveragePooling1D, Embedding, LSTM, Dropout
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.optimizers import Adam
import os
pd.options.mode.chained_assignment = None
np.random.seed(2017)

Using TensorFlow backend.
/home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
train_data = pd.read_csv('../../data/spooky-author/download/train.csv')
test_data = pd.read_csv('../../data/spooky-author/download/test.csv')
mapper = {'EAP':0, 'HPL':1, 'MWS':2}
train_data['author'] = train_data['author'].map(lambda x : mapper[x])

In [3]:
train_texts = train_data['text'].values.tolist()
score_texts = test_data['text'].values.tolist()
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_texts + score_texts)
train_sequences = tokenizer.texts_to_sequences(train_texts)
test_sequences = tokenizer.texts_to_sequences(score_texts)
word_index = tokenizer.word_index
print('words:', len(word_index))
train_seq = pad_sequences(train_sequences, maxlen=90)
score_seq = pad_sequences(test_sequences, maxlen=90)
labels = to_categorical(train_data['author'].values.tolist())
print('data:', train_seq.shape, score_seq.shape)
print('label:', labels.shape)

words: 29451
data: (19579, 90) (8392, 90)
label: (19579, 3)


### train model

In [4]:
indices = np.arange(train_seq.shape[0])
np.random.shuffle(indices)
train_seq = train_seq[indices]
labels = labels[indices]
nb_validation_samples = int(0.15 * train_seq.shape[0])
x_train = train_seq[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = train_seq[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

In [5]:
embeddings_index = {}

f = open(os.path.join('../../src/pip/glove/glove.6B.50d.txt'))

for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

embedding_matrix = np.zeros((len(word_index) + 1, 50))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

del embeddings_index

embedding_layer = Embedding(len(word_index) + 1, 50, weights=[embedding_matrix], input_length=90, trainable=True)

In [6]:
model = Sequential()
model.add(embedding_layer)
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dropout(0.2))
model.add(Dense(3, activation='softmax'))
optimizer = Adam(lr=0.0001)
model.compile(loss='categorical_crossentropy',optimizer=optimizer,metrics=['accuracy'])

In [7]:
params = {}
params['validation_data'] = (x_val, y_val)
params['batch_size'] = 8
params['epochs'] = 3
params['callbacks'] = [EarlyStopping(patience=2, monitor='val_loss')]
model.optimizer.lr = 0.0001
model.fit(x_train, y_train, **params, )

Train on 16643 samples, validate on 2936 samples
Epoch 1/3
16643/16643 [==============================] - 160s 10ms/step - loss: 1.0072 - acc: 0.4932 - val_loss: 0.8963 - val_acc: 0.5841
Epoch 2/3
16643/16643 [==============================] - 159s 10ms/step - loss: 0.8896 - acc: 0.5904 - val_loss: 0.8265 - val_acc: 0.6308
Epoch 3/3
16643/16643 [==============================] - 158s 9ms/step - loss: 0.8305 - acc: 0.6333 - val_loss: 0.7721 - val_acc: 0.6580


In [8]:
params = {}
params['validation_data'] = (x_val, y_val)
params['batch_size'] = 8
params['epochs'] = 3
params['callbacks'] = [EarlyStopping(patience=2, monitor='val_loss')]
model.optimizer.lr = 0.001
model.fit(x_train, y_train, **params)

Train on 16643 samples, validate on 2936 samples
Epoch 1/3
16643/16643 [==============================] - 155s 9ms/step - loss: 0.7846 - acc: 0.6608 - val_loss: 0.7148 - val_acc: 0.6958
Epoch 2/3
16643/16643 [==============================] - 155s 9ms/step - loss: 0.7325 - acc: 0.6886 - val_loss: 0.6769 - val_acc: 0.7207
Epoch 3/3
16643/16643 [==============================] - 155s 9ms/step - loss: 0.6934 - acc: 0.7121 - val_loss: 0.6472 - val_acc: 0.7333


In [9]:
params = {}
params['validation_data'] = (x_val, y_val)
params['batch_size'] = 16
params['epochs'] = 10
params['callbacks'] = [EarlyStopping(patience=2, monitor='val_loss')]
model.optimizer.lr = 0.0005
model.fit(x_train, y_train, **params)

Train on 16643 samples, validate on 2936 samples
Epoch 1/10
16643/16643 [==============================] - 77s 5ms/step - loss: 0.6490 - acc: 0.7307 - val_loss: 0.6309 - val_acc: 0.7384
Epoch 2/10
16643/16643 [==============================] - 77s 5ms/step - loss: 0.6300 - acc: 0.7442 - val_loss: 0.6128 - val_acc: 0.7480
Epoch 3/10
16643/16643 [==============================] - 77s 5ms/step - loss: 0.5974 - acc: 0.7595 - val_loss: 0.5890 - val_acc: 0.7629
Epoch 4/10
16643/16643 [==============================] - 78s 5ms/step - loss: 0.5793 - acc: 0.7681 - val_loss: 0.5812 - val_acc: 0.7650
Epoch 5/10
16643/16643 [==============================] - 77s 5ms/step - loss: 0.5497 - acc: 0.7779 - val_loss: 0.5608 - val_acc: 0.7759
Epoch 6/10
16643/16643 [==============================] - 77s 5ms/step - loss: 0.5320 - acc: 0.7884 - val_loss: 0.5541 - val_acc: 0.7725
Epoch 7/10
16643/16643 [==============================] - 78s 5ms/step - loss: 0.5105 - acc: 0.7975 - val_loss: 0.5411 - val_acc:

In [11]:
params = {}
params['validation_data'] = (x_val, y_val)
params['batch_size'] = 16
params['epochs'] = 10
params['callbacks'] = [EarlyStopping(patience=2, monitor='val_loss')]
model.optimizer.lr = 0.0003
model.fit(x_train, y_train, **params)

Train on 16643 samples, validate on 2936 samples
Epoch 1/10
16643/16643 [==============================] - 78s 5ms/step - loss: 0.4110 - acc: 0.8405 - val_loss: 0.5033 - val_acc: 0.8048
Epoch 2/10
16643/16643 [==============================] - 78s 5ms/step - loss: 0.3920 - acc: 0.8467 - val_loss: 0.4943 - val_acc: 0.8069
Epoch 3/10
16643/16643 [==============================] - 77s 5ms/step - loss: 0.3727 - acc: 0.8581 - val_loss: 0.4943 - val_acc: 0.8113
Epoch 4/10
16643/16643 [==============================] - 78s 5ms/step - loss: 0.3521 - acc: 0.8670 - val_loss: 0.4800 - val_acc: 0.8110
Epoch 5/10
16643/16643 [==============================] - 78s 5ms/step - loss: 0.3448 - acc: 0.8709 - val_loss: 0.4835 - val_acc: 0.8147
Epoch 6/10
16643/16643 [==============================] - 78s 5ms/step - loss: 0.3326 - acc: 0.8749 - val_loss: 0.4734 - val_acc: 0.8154
Epoch 7/10
16643/16643 [==============================] - 78s 5ms/step - loss: 0.3169 - acc: 0.8817 - val_loss: 0.4777 - val_acc:

In [12]:
params = {}
params['validation_data'] = (x_val, y_val)
params['batch_size'] = 32
params['epochs'] = 5
params['callbacks'] = [EarlyStopping(patience=2, monitor='val_loss')]
model.optimizer.lr = 0.0002
model.fit(x_train, y_train, **params)

Train on 16643 samples, validate on 2936 samples
Epoch 1/5
16643/16643 [==============================] - 39s 2ms/step - loss: 0.2600 - acc: 0.9015 - val_loss: 0.4723 - val_acc: 0.8273
Epoch 2/5
16643/16643 [==============================] - 39s 2ms/step - loss: 0.2567 - acc: 0.9064 - val_loss: 0.4716 - val_acc: 0.8246
Epoch 3/5
16643/16643 [==============================] - 39s 2ms/step - loss: 0.2467 - acc: 0.9089 - val_loss: 0.4714 - val_acc: 0.8266
Epoch 4/5
16643/16643 [==============================] - 39s 2ms/step - loss: 0.2381 - acc: 0.9119 - val_loss: 0.4730 - val_acc: 0.8273
Epoch 5/5
16643/16643 [==============================] - 39s 2ms/step - loss: 0.2340 - acc: 0.9134 - val_loss: 0.4800 - val_acc: 0.8266


### score model

In [13]:
train_predict = pd.DataFrame(model.predict_proba(train_seq), columns=['l1','l2','l3'])
train_class = pd.DataFrame(model.predict_classes(train_seq), columns=['lstm'])
test_predict = pd.DataFrame(model.predict_proba(score_seq), columns=['l1','l2','l3'])
test_class = pd.DataFrame(model.predict_classes(score_seq), columns=['lstm'])
train_keras = pd.concat([train_data[['id']], train_predict, train_class], axis=1)
test_keras = pd.concat([test_data[['id']], test_predict, test_class], axis=1)
print('train',train_keras.shape)
print('test', test_keras.shape)
train_keras.to_csv('../../data/spooky-author/data/train_lstm_score.csv', index=False)
test_keras.to_csv('../../data/spooky-author/data/test_lstm_score.csv', index=False)

train (19579, 5)
test (8392, 5)
